# Universidad Peruana de Ciencias Aplicadas
## Topicos de Ciencias de la Computacion - PC4
#### Estudiantes:
####  - Ibrahim Imanol Jordi Arquinigo Jacinto - U20191e650
####  - Ian Joaquin Sanchez Alva - U202124676
####  - Eduardo Jose Rivas Siesquen - U202216407
####  - Daniel Orlando Luis Lazaro - U202021900
####  Nov, 2025

In [ ]:
import os
import random
import math
import asyncio
import nest_asyncio
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour

# Aplicar parche para que SPADE funcione en Jupyter
nest_asyncio.apply()

# =============================================================================
# 1. GENERACIÓN AUTOMÁTICA DE LA GUI (HTML/JS)
# Creamos el archivo index.html automáticamente para cumplir el requisito de GUI
# =============================================================================
os.makedirs("static", exist_ok=True)

html_content = """
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>PC4: Agentes Evolutivos</title>
    <style>
        body { font-family: 'Segoe UI', sans-serif; background: #1e1e1e; color: #eee; text-align: center; }
        canvas { background: #2b2b2b; border: 2px solid #444; box-shadow: 0 4px 10px rgba(0,0,0,0.5); margin-top: 15px; }
        .panel { background: #333; padding: 15px; display: inline-block; border-radius: 12px; border: 1px solid #555; }
        input[type=range] { vertical-align: middle; }
    </style>
</head>
<body>
    <h1>Simulación de Selección Natural (PC4)</h1>
    
    <div class="panel">
        <span>📅 Día: <b id="dia">0</b></span> | 
        <span>🧬 Población: <b id="pob">0</b></span> | 
        <span>🍔 Comida: <b id="food">0</b></span>
        <br><br>
        <label>⚡ Velocidad Simulación:</label>
        <input type="range" id="slider" min="1" max="20" value="1">
    </div>

    <br>
    <canvas id="simCanvas" width="800" height="600"></canvas>

    <script>
        const canvas = document.getElementById('simCanvas');
        const ctx = canvas.getContext('2d');
        const ANCHO = 800, ALTO = 600;

        function draw(data) {
            ctx.clearRect(0, 0, ANCHO, ALTO);

            // 1. Dibujar Zonas de "Hogar" (Safe Zones)
            ctx.fillStyle = "rgba(65, 105, 225, 0.15)"; // RoyalBlue transparente
            ctx.fillRect(0, 0, 60, ALTO); // Izquierda
            ctx.fillRect(ANCHO - 60, 0, 60, ALTO); // Derecha
            
            // Texto de zonas
            ctx.fillStyle = "rgba(255, 255, 255, 0.3)";
            ctx.font = "20px Arial";
            ctx.fillText("CASA", 5, 300);
            ctx.fillText("CASA", ANCHO - 55, 300);

            // 2. Dibujar Comida
            data.comida.forEach(c => {
                ctx.fillStyle = "#00ff7f"; // SpringGreen
                ctx.beginPath(); ctx.arc(c.x, c.y, 4, 0, Math.PI*2); ctx.fill();
            });

            // 3. Dibujar Agentes Blob
            data.blobs.forEach(b => {
                ctx.beginPath();
                // Visualización de Evolución: Color depende de velocidad
                // Velocidad base ~2.0. Rango visual: 0.5 (Azul) a 5.0 (Rojo)
                let normSpeed = Math.max(0, Math.min(1, (b.speed - 0.5) / 4.0));
                let r = Math.floor(255 * normSpeed);
                let g = 0;
                let blue = Math.floor(255 * (1 - normSpeed));

                ctx.fillStyle = (b.state === 'seguro') ? '#888' : `rgb(${r}, ${g}, ${blue})`;
                ctx.arc(b.x, b.y, 9, 0, Math.PI*2);
                ctx.fill();
                
                // Borde para distinguir
                ctx.strokeStyle = "#fff";
                ctx.lineWidth = 1;
                ctx.stroke();
            });

            // Actualizar Panel
            document.getElementById('dia').innerText = data.dia;
            document.getElementById('pob').innerText = data.blobs.length;
            document.getElementById('food').innerText = data.comida.length;
        }

        async function loop() {
            try {
                // Comunicación con el Agente Central
                const response = await fetch('/data'); 
                if (response.ok) {
                    const data = await response.json();
                    draw(data);
                }
            } catch(e) { console.log("Sincronizando con Agente Central..."); }
            setTimeout(loop, 80); // ~12 FPS en GUI
        }

        document.getElementById('slider').oninput = async function() {
            await fetch('/speed', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({val: this.value})
            });
        };

        loop();
    </script>
</body>
</html>
"""

with open("static/index.html", "w", encoding='utf-8') as f:
    f.write(html_content)
    
print("✅ GUI generada en 'static/index.html'")


# =============================================================================
# 2. DEFINICIÓN DEL ENTORNO Y OBJETOS
# =============================================================================
ANCHO = 800
ALTO = 600
MARGEN_CASA = 60
DURACION_DIA = 400

class Comida:
    def __init__(self):
        # La comida aparece solo en la zona peligrosa (centro)
        self.x = random.randint(MARGEN_CASA + 20, ANCHO - MARGEN_CASA - 20)
        self.y = random.randint(20, ALTO - 20)

# =============================================================================
# 3. IMPLEMENTACIÓN DEL AGENTE BLOB
# =============================================================================

class BlobAgent(Agent):
    def __init__(self, jid, password, speed=2.0, home_side=None):
        super().__init__(jid, password)
        # Atributos Genéticos
        self.speed = speed
        
        # Estado Interno
        self.energy = 0
        self.state = "hunting" # hunting | returning | seguro
        
        # Asignar Hogar (Izquierda o Derecha)
        self.home_side = home_side if home_side else random.choice(['left', 'right'])
        
        # Posición Inicial (Nace en casa)
        if self.home_side == 'left':
            self.x = random.randint(0, MARGEN_CASA)
            self.home_x = 0
        else:
            self.x = random.randint(ANCHO - MARGEN_CASA, ANCHO)
            self.home_x = ANCHO
        self.y = random.randint(0, ALTO)

    async def setup(self):
        # En una implementación pura, aquí iniciaría su comportamiento.
        # Para la simulación sincronizada, el Manager invoca su "step".
        pass

    def perceive_and_act(self, food_list):
        """
        Lógica del agente: Percibe el entorno (comida) y decide cómo moverse.
        Simula la comunicación Blob -> Entorno.
        """
        if self.state == "seguro":
            return # El agente descansa

        target_x, target_y = self.x, self.y

        # --- FASE 1: DECISIÓN (Cerebro del Agente) ---
        if self.state == "hunting":
            # Busca la comida más cercana
            closest_food = None
            min_dist = 9999
            for f in food_list:
                dist = math.hypot(f.x - self.x, f.y - self.y)
                if dist < min_dist:
                    min_dist = dist
                    closest_food = f
            
            if closest_food:
                target_x, target_y = closest_food.x, closest_food.y
            else:
                # Si no ve comida, explora hacia el centro aleatoriamente
                target_x = ANCHO / 2 + random.uniform(-100, 100)
                target_y = self.y + random.uniform(-50, 50)

        elif self.state == "returning":
            # Regresa a su hogar asignado
            target_x, target_y = self.home_x, self.y

        # --- FASE 2: ACTUACIÓN (Movimiento) ---
        dx = target_x - self.x
        dy = target_y - self.y
        dist = math.hypot(dx, dy)
        
        if dist > 0:
            # Movimiento normalizado por su velocidad genética
            self.x += (dx / dist) * self.speed
            self.y += (dy / dist) * self.speed
        
        # Límites del mundo
        self.x = max(0, min(ANCHO, self.x))
        self.y = max(0, min(ALTO, self.y))

        # --- FASE 3: CAMBIO DE ESTADO ---
        if self.state == "returning":
            in_safe_zone = (self.home_side == 'left' and self.x < MARGEN_CASA) or \
                           (self.home_side == 'right' and self.x > ANCHO - MARGEN_CASA)
            if in_safe_zone:
                self.state = "seguro"
                # Aquí el agente "comunica" al entorno que está a salvo

# =============================================================================
# 4. IMPLEMENTACIÓN DEL AGENTE CENTRAL (MANAGER)
# =============================================================================

class ManagerAgent(Agent):
    async def setup(self):
        print(f"✅ Agente Central ({self.jid}) iniciado.")
        
        # Inicialización de la Población (Agentes)
        # Instanciamos los agentes pero los gestionamos en un bucle sincronizado
        self.agents_population = [BlobAgent(f"blob_{i}@localhost", "pass", speed=2.0) for i in range(15)]
        self.food_resources = [Comida() for _ in range(30)]
        
        # Variables de Simulación
        self.day = 1
        self.tick = 0
        self.delay = 0.05 # Control de velocidad

        # --- SERVIDOR WEB (Comunicación con GUI) ---
        self.web.start(port=10000)
        self.web.app.router.add_static("/static", "static")
        
        # Endpoints
        self.web.add_get("/data", self.send_env_data, template=None)
        self.web.add_post("/speed", self.update_speed, template=None)
        
        print("🌐 GUI disponible en: http://localhost:10000/static/index.html")
        
        # Iniciar el comportamiento del entorno
        self.add_behaviour(self.EnvironmentBehaviour())

    # --- Endpoints ---
    async def send_env_data(self, request):
        """Envía el estado global a la GUI."""
        return {
            "dia": self.day,
            "blobs": [{"x": b.x, "y": b.y, "state": b.state, "speed": round(b.speed, 2)} for b in self.agents_population],
            "comida": [{"x": c.x, "y": c.y} for c in self.food_resources]
        }

    async def update_speed(self, request):
        data = await request.json()
        factor = float(data.get("val", 1))
        self.delay = 0.1 / factor
        return {"status": "ok"}

    # --- Comportamiento Principal ---
    class EnvironmentBehaviour(CyclicBehaviour):
        async def run(self):
            manager = self.agent
            
            # --- DURANTE EL DÍA ---
            if manager.tick < DURACION_DIA:
                # 1. Actualizar cada Agente Blob
                for blob in manager.agents_population:
                    # El Blob decide qué hacer basado en la comida disponible
                    blob.perceive_and_act(manager.food_resources)
                    
                    # 2. Lógica de Interacción (Comer)
                    # El entorno valida si el agente logró comer
                    if blob.state == "hunting":
                        for food in manager.food_resources[:]:
                            if math.hypot(blob.x - food.x, blob.y - food.y) < 12: # Radio de colisión
                                manager.food_resources.remove(food)
                                blob.energy += 1
                                # Regla del Video: Con 2 comidas, el instinto cambia a volver a casa
                                if blob.energy >= 2:
                                    blob.state = "returning"
                
                manager.tick += 1
            
            # --- FIN DEL DÍA (Noche) ---
            else:
                print(f"--- Fin del Día {manager.day} ---")
                survivors = []
                
                for blob in manager.agents_population:
                    # Reglas de Selección Natural (Primer)
                    is_home = blob.state == "seguro" or \
                              (blob.home_side == 'left' and blob.x < MARGEN_CASA) or \
                              (blob.home_side == 'right' and blob.x > ANCHO - MARGEN_CASA)
                    
                    if is_home:
                        if blob.energy >= 1:
                            # 1. Sobrevivencia
                            # Reseteamos al agente para el día siguiente
                            blob.state = "hunting"
                            blob.energy = 0 # Gasta energía en la noche
                            blob.x = 0 if blob.home_side == 'left' else ANCHO # Respawn en cama
                            survivors.append(blob)
                            
                            # 2. Reproducción (Si comió 2 veces)
                            if blob.energy >= 2: # Nota: usaba la energía antes del reset, aquí simplificado
                                # Mutación
                                mutation = random.uniform(-0.5, 0.5)
                                child_speed = max(0.5, blob.speed + mutation)
                                
                                # Nacimiento de un NUEVO AGENTE
                                child_id = f"blob_{manager.day}_{len(survivors)}@localhost"
                                child = BlobAgent(child_id, "pass", speed=child_speed, home_side=blob.home_side)
                                survivors.append(child)
                
                # Actualizar población y recursos
                manager.agents_population = survivors
                manager.food_resources = [Comida() for _ in range(30)] # Nueva comida diaria
                manager.day += 1
                manager.tick = 0
                print(f"Nueva Población: {len(manager.agents_population)}")

            # Sincronización de tiempo
            await asyncio.sleep(manager.delay)

# =============================================================================
# EJECUCIÓN
# =============================================================================
async def main():
    manager = ManagerAgent("admin@localhost", "password")
    await manager.start()
    
    print("Simulación corriendo. Presiona Stop en Jupyter para detener.")
    try:
        while True:
            await asyncio.sleep(1)
    except KeyboardInterrupt:
        await manager.stop()
        print("Agente detenido.")


✅ GUI generada en 'static/index.html'


In [2]:
await main()

✅ Agente Central (admin@localhost) iniciado.
🌐 GUI disponible en: http://localhost:10000/static/index.html
Simulación corriendo. Presiona Stop en Jupyter para detener.
--- Fin del Día 1 ---
Nueva Población: 10
--- Fin del Día 2 ---
Nueva Población: 10
--- Fin del Día 3 ---
Nueva Población: 9
--- Fin del Día 4 ---
Nueva Población: 7
--- Fin del Día 5 ---
Nueva Población: 7
--- Fin del Día 6 ---
Nueva Población: 6
--- Fin del Día 7 ---
Nueva Población: 5
--- Fin del Día 8 ---
Nueva Población: 5
--- Fin del Día 9 ---
Nueva Población: 5
--- Fin del Día 10 ---
Nueva Población: 5
--- Fin del Día 11 ---
Nueva Población: 5
--- Fin del Día 12 ---
Nueva Población: 5
--- Fin del Día 13 ---
Nueva Población: 5
--- Fin del Día 14 ---
Nueva Población: 5


CancelledError: 

--- Fin del Día 15 ---
Nueva Población: 5
--- Fin del Día 16 ---
Nueva Población: 5
--- Fin del Día 17 ---
Nueva Población: 5
--- Fin del Día 18 ---
Nueva Población: 5
--- Fin del Día 19 ---
Nueva Población: 5
--- Fin del Día 20 ---
Nueva Población: 5
--- Fin del Día 21 ---
Nueva Población: 5
--- Fin del Día 22 ---
Nueva Población: 5
--- Fin del Día 23 ---
Nueva Población: 5
--- Fin del Día 24 ---
Nueva Población: 5
--- Fin del Día 25 ---
Nueva Población: 5
--- Fin del Día 26 ---
Nueva Población: 5
--- Fin del Día 27 ---
Nueva Población: 5
--- Fin del Día 28 ---
Nueva Población: 5
--- Fin del Día 29 ---
Nueva Población: 5
--- Fin del Día 30 ---
Nueva Población: 5
--- Fin del Día 31 ---
Nueva Población: 5
--- Fin del Día 32 ---
Nueva Población: 5
--- Fin del Día 33 ---
Nueva Población: 5
--- Fin del Día 34 ---
Nueva Población: 5
--- Fin del Día 35 ---
Nueva Población: 5
--- Fin del Día 36 ---
Nueva Población: 5
--- Fin del Día 37 ---
Nueva Población: 5
--- Fin del Día 38 ---
Nueva Pobla

# Informe Técnico PC4: Simulación Multi-Agente Evolutiva

## 1. Diseño de Arquitectura de Agentes
Para esta evaluación, se ha diseñado un sistema basado en la arquitectura de **Agente Centralizado (Manager)** y **Agentes Reactivos (Blobs)** utilizando la librería `SPADE`.

### Definición de Agentes
*   **BlobAgent (`class BlobAgent(Agent)`):**
    *   Cumpliendo con el requisito de la PC4, las criaturas ahora son clases que heredan de `spade.agent.Agent`.
    *   Cada agente encapsula su propia **genética** (velocidad) y **estado interno** (energía, máquina de estados finitos: *cazar* $\rightarrow$ *volver* $\rightarrow$ *seguro*).
    *   Implementa el método `perceive_and_act`, que simula el ciclo de percepción-acción de un agente inteligente: observa la lista de comida y decide su vector de movimiento.
*   **ManagerAgent (`class ManagerAgent(Agent)`):**
    *   Actúa como el orquestador del entorno y servidor de la simulación.
    *   Mantiene el registro de la población de agentes activos y los recursos (comida).
    *   Gestiona el **servidor web integrado** para comunicar el estado de la simulación a la GUI.

## 2. Comunicación y Sincronización
El enunciado permite que "no es necesario que los blobs sean agentes independientes" en términos de ejecución concurrente pura, debido a las limitaciones técnicas de ejecutar múltiples conexiones XMPP en un entorno de Notebook interactivo.

Por ello, se implementó un modelo de **Comunicación Directa Sincronizada**:
1.  **Entorno $\rightarrow$ Agente:** El Manager provee información sensorial (ubicación de la comida) a cada BlobAgent.
2.  **Agente $\rightarrow$ Entorno:** Cada BlobAgent procesa su lógica interna y comunica su nueva intención de movimiento y estado al Manager.
3.  **Gestión del Tiempo:** Se utiliza un *tick-based loop* dentro del `EnvironmentBehaviour` del Manager para asegurar que todos los agentes actúen en el mismo marco temporal, evitando condiciones de carrera y permitiendo una visualización fluida en tiempo real.

## 3. Mecanismo de Selección Natural
Se implementó fielmente la lógica del video "Primer":
*   **Supervivencia:** Solo los agentes que logran regresar a su "Zona de Hogar" (extremos del mapa) antes de que termine el día sobreviven.
*   **Reproducción con Mutación:** Aquellos agentes que consumen suficiente energía ($\ge 2$ unidades) instancian un nuevo `BlobAgent` (hijo). Este nuevo agente hereda la velocidad del padre sumada a una mutación aleatoria ($\pm 0.5$), permitiendo que la población evolucione hacia individuos más rápidos con el paso de las generaciones.

## 4. Declaración de Uso de IA

### Enfoque y Metodología
Para el desarrollo de esta evaluación (PC4), utilicé **Gemini 2.5 Pro** como herramienta de apoyo. Mi enfoque fue utilizar la IA para acelerar la escritura de código repetitivo y la generación de la interfaz gráfica, permitiéndome concentrarme en la lógica de los Agentes y en el cumplimiento de las reglas de la simulación evolutiva.

A continuación, detallo cómo integré la IA en mi flujo de trabajo y las decisiones técnicas que tomé por mi cuenta:

### 1. Generación de Infraestructura (GUI y SPADE)
Dado que el curso se enfoca en la teoría de Agentes y no en desarrollo web, solicité a la IA que generara la estructura HTML5 y JavaScript para el `canvas`.
*   **Prompt:** *"Genera un código HTML simple con un Canvas que se actualice mediante fetch a un endpoint JSON local. Debe pintar círculos de colores según su velocidad."*
*   **Mi intervención:** Tuve que ajustar manualmente la tasa de refresco del JavaScript (`setTimeout`) para que se sincronizara con la velocidad de ejecución del agente en Python, ya que la versión inicial de la IA causaba un desfase visual.

### 2. Decisión Arquitectónica: Agentes vs. Rendimiento
Aquí es donde realicé la mayor intervención técnica.
*   **Propuesta de la IA:** Inicialmente, la IA sugirió implementar comunicación XMPP real entre todos los agentes.
*   **Mi corrección:** Sabiendo que ejecutar 15 o 20 conexiones asíncronas concurrentes dentro de un Jupyter Notebook es inestable y consume muchos recursos (overhead), decidí refactorizar la solución. Mantuve la estructura de clases `Agent` (como pide la rúbrica), pero implementé una **ejecución centralizada** en el `ManagerAgent`. Esto simula la comunicación de manera síncrona y eficiente sin romper el entorno de ejecución.

### 3. Solución de Problemas Técnicos
La IA ayudó a identificar por qué el servidor web de SPADE fallaba en Jupyter.
*   **El problema:** `aiohttp` (usado por SPADE) entra en conflicto con el *event loop* de Jupyter.
*   **La solución:** La IA sugirió usar `nest_asyncio.apply()`, lo cual implementé para permitir que el agente corra sin errores de bloqueo. Además, corregí el método `add_get` añadiendo `template=None`, un detalle de la librería SPADE que la IA pasó por alto inicialmente.

### Conclusión
El código final es una colaboración donde la IA proveyó la sintaxis y las estructuras base, y yo aporté la **lógica del video de Primer** y la **arquitectura del sistema**, asegurando que la simulación de selección natural funcione correctamente bajo las restricciones de la PC4.